In [ ]:
#default_exp utility

In [ ]:
#export

import json
import pandas as pd

In [ ]:
#no_test
'''
names = pd.read_csv("names.csv")
names.head()

name_list = names["name"].tolist()
genders = names["gender"].tolist()
languages = names["language"].tolist()
types = names["type"].tolist()

names = []

for i in range(len(name_list)):
    name = {}
    name["name"] = name_list[i]
    name["sex"] = genders[i]
    name["language"] = languages[i]
    name["type"] = types[i]
    names.append(name)
    
with open("names.json", "w") as outfile:
    outfile.write('{\n\"names\": \n')
    first_name = True
    for name in names:
        if first_name:
            first_name = False
        else:
            outfile.write(",\n")
        json.dump(name, outfile)
    outfile.write("\n]\n")
    outfile.write('}')
'''

'\nnames = pd.read_csv("names.csv")\nnames.head()\n\nname_list = names["name"].tolist()\ngenders = names["gender"].tolist()\nlanguages = names["language"].tolist()\ntypes = names["type"].tolist()\n\nnames = []\n\nfor i in range(len(name_list)):\n    name = {}\n    name["name"] = name_list[i]\n    name["sex"] = genders[i]\n    name["language"] = languages[i]\n    name["type"] = types[i]\n    names.append(name)\n    \nwith open("names.json", "w") as outfile:\n    outfile.write(\'{\n"names": \n\')\n    first_name = True\n    for name in names:\n        if first_name:\n            first_name = False\n        else:\n            outfile.write(",\n")\n        json.dump(name, outfile)\n    outfile.write("\n]\n")\n    outfile.write(\'}\')\n'

In [ ]:
#export

def compact_references(people, events):
    '''
    eliminates aggregated event-specific IDs and adds a references field to people record
        people: a list of dictionaries in which each element is a disambiguated person record
        events: a list of dictionaries in which each element is an event record
        
        returns: modified people and events list containing single unique identifier for each individual
    '''
    
    old_ids = []
    new_ids = []
    volume = people[0]["id"][:people[0]["id"].find('-')]
    
    for index, person in enumerate(people):
        person["references"] = []
        new_id = volume + '-P' + '0' * (5 - len(str(index + 1))) + str(index + 1)
        for eyedee in person["id"].split(';'):
            if not (eyedee[:eyedee.rfind('-')] in person["references"]):
                person["references"].append(eyedee[:eyedee.rfind('-')])
            old_ids.append(eyedee)
            new_ids.append(new_id)
        person["id"] = new_id
        
    for person in people:
        if person["relationships"] != None:
            for relationship in person["relationships"]:
                relationship["related_person"] = new_ids[old_ids.index(relationship["related_person"])]
            
    for event in events:
        if event["principal"] != None:
            event["principal"] = new_ids[old_ids.index(event["principal"])]
        if event["cleric"] != None:
            event["cleric"] = new_ids[old_ids.index(event["cleric"])]
            
    return people, events

In [ ]:
#export

def determine_sex(name, name_list="names.json", language=None):
    '''
    determines the most likely sex of a name based on a specified controlled vocabulary
        name: target name
        name_list: path to json file containing name controlled vocabulary
        language: specific language, if required for disambiguation
        
        returns: predicted sex as a string, or "unknown" if no prediction possible
    '''
    with open(name_list, encoding="utf-8") as infile:
        name_file = json.load(infile)
        
    names = name_file["names"]
    
    for i in range(len(names)):
        if (name == names[i]["name"]) and ((language == None) or (language == names[i]["language"])):
            return names[i]["sex"]
        
    return "unknown"

In [ ]:
#export

def new_file(files_list):
    with open('new_file.xml', 'w', encoding = "utf-8") as outfile:
        for names in files_list:
            with open(names, encoding = "utf-8") as infile:
                outfile.write(infile.read())
            outfile.write("\n")
            
    return

In [ ]:
#export

def retrieve_json_vocab(path_to_dict, vocab_type, language=None):
    '''
    retrieves specified controlled vocabulary from a specified json file
        path_to_dict: path to a JSON file containing controlled vocabulary
        vocab_type: type of vocabulary, corresponding to label in JSON
        language: specific non-English language, if applicable
        
        returns: controlled vocabulary, as a list
    '''
    
    with open(path_to_dict, encoding="utf-8") as infile:
        synonyms = json.load(infile)
        
    for vocab in synonyms["controlled_vocabularies"]:
        if language == None:
            if vocab["type"] == vocab_type:
                return vocab["terms"]
        else:
            if (vocab["type"] == vocab_type) and ("language" in vocab) and (vocab["language"] == language):
                return vocab["terms"] 
        
    return "unable to locate vocab"

In [ ]:
#export

def normalize_text(text, path_to_dict, context=None):
    '''
    normalizes transcribed text by expanding abbreviations, correcting idiosyncratic spellings, etc.
        text: text to be normalized, as a string
        path_to_dict: path to a JSON file containing baseword-synonym key-value pairs
        context: specific normalization context, if applicable
    
        returns: normalized text, as a string
    '''
    with open(path_to_dict, encoding="utf-8") as infile:
        synonyms = json.load(infile)
        
    for word in synonyms["words"]:        
        if (context != None) and ("context" in word) and (context not in word["context"]):            
            continue
        for synonym in word["synonyms"]:            
            text = (' ' + text + ' ').replace(' ' + synonym + ' ', ' ' + word["baseword"] + ' ')
            text = text[1:len(text) - 1]
    
    return text

In [ ]:
#export

def translate_characteristic(characteristic, path_to_dict, language):
    '''
    translates characteristics from non-English languages to standardized English equivalent
        characteristic: characteristic to be translated, as a string
        path_to_dict: path to a JSON file containing language equivalent key-value pairs
        language: language to be translated from
    
        returns: translated characteristic, as a string
    '''
    
    with open(path_to_dict, encoding="utf-8") as infile:
        synonyms = json.load(infile)
        
    for word in synonyms["translations"]:        
        for translation in word[language]:            
            if translation == characteristic:                
                return word["english"]
    
    return characteristic

In [ ]:
#export

def compare_dates(year1, month1, day1, year2, month2, day2):
    if year1 > year2:
        return '>'
    elif year1 < year2:
        return '<'
    elif month1 > month2:
        return '>'
    elif month1 < month2:
        return '<'
    elif day1 > day2:
        return '>'
    elif day1 < day2:
        return '<'
    else:
        return '='

In [ ]:
#export

def complete_date(date, pvs_date, subs_date):    
    
    year = date[:4]    
    month = date[5:7]    
    day = date[8:]
        
    if pvs_date == None:
        subs_year = subs_date[:4]
        subs_month = subs_date[5:7]
        subs_day = subs_date[8:]
        if year == "????":
            return "/" + subs_date
        else:
            if month == "??":
                if day == "??":
                    if year == subs_year:
                        return year + '-01-01/' + subs_date
                    else:
                        return year + '-01-01/' + year + '-12-31'
                else:
                    if year == subs_year:
                        return year + '-01' + day + '/' + subs_date
                    else:
                        return year + '-01' + day + '/' + year + '-12-' + day                    
            else:
                next_month = str(int(month) + 1)
                if len(next_month) == 1:
                    next_month = '0' + next_month
                return year + '-' + month + '-01/' + year + '-' + next_month + '-01'
    elif subs_date == None:
        pvs_year = pvs_date[:4]
        pvs_month = pvs_date[5:7]
        pvs_day = pvs_date[8:]
        if year == "????":
            if month == "??":
                if day == "??":
                    return pvs_date + '/'
                elif day < pvs_day:
                    if pvs_month == "12":
                        year = str(int(pvs_year) + 1)                        
                        return year + '-01-' + day + '/'
                    else:
                        month = str(int(pvs_month) + 1)
                        if len(month) == 1:
                            month = '0' + month
                        return pvs_year + '-' + month + '-' + day + '/'
                else:
                    return pvs_year + '-' + pvs_month + '-' + day + '/'
            elif month < pvs_month:
                year = str(int(pvs_year) + 1)
                if day == "??":                    
                    return year + '-' + month + '-01/'
                else:
                    return year + '-' + month + '-' + day + '/'
            else:
                if day == "??":
                    return pvs_year + '-' + month + '-01/'
                else:
                    return pvs_year + '-' + month + '-' + day + '/'
        elif month != "??":
            next_month = str(int(month) + 1)
            if len(next_month) == 1:
                next_month = '0' + next_month
            return year + '-' + month + '-01/' + year + '-' + next_month + '-01'
        elif day != "??":
            if year == pvs_year:
                return year + '-' + pvs_month + '-' + day + '/' + year + '-12-' + day
            else:
                return year + '-01-' + day + '/' + year + '-12-' + day
        else:
            if year == pvs_year:
                return year + '-' + pvs_month + '-' + pvs_day + '/' + year + '-12-31'
            else:
                return year + '-01-01/' + year + '-12-31'
    else:
        subs_year = subs_date[:4]
        subs_month = subs_date[5:7]
        subs_day = subs_date[8:]
        pvs_year = pvs_date[:4]
        pvs_month = pvs_date[5:7]
        pvs_day = pvs_date[8:]
        
        if (subs_year == pvs_year) and (subs_month == pvs_month) and (subs_day == pvs_day):
            return pvs_year + '-' + pvs_month + '-' + pvs_day        
                
        if year == "????":
            if month == "??":
                if day == "??":
                    return pvs_date + '/' + subs_date               
                elif (int(day) >= int(pvs_day)) and (int(day) <= int(subs_day)):
                    return pvs_year + '-' + pvs_month + '-' + day + '/' + subs_year + '-' + subs_month + '-' + day
                elif (int(day) >= int(pvs_day)):
                    subs_month = str(int(subs_month) - 1)
                    if len(subs_month) == 1:
                        subs_month = '0' + subs_month
                    if subs_month == "00":
                        subs_month = "12"
                        subs_year = str(int(subs_year) - 1)
                    return pvs_year + '-' + pvs_month + '-' + day + '/' + subs_year + '-' + subs_month + '-' + day
                elif (int(day) <= int(subs_day)):
                    pvs_month = str(int(pvs_month) + 1)
                    if len(pvs_month) == 1:
                        pvs_month = '0' + pvs_month
                    if pvs_month == "13":
                        pvs_month = "01"
                        pvs_year = str(int(pvs_year) + 1)
                    return pvs_year + '-' + pvs_month + '-' + day + '/' + subs_year + '-' + subs_month + '-' + day
                else:
                    subs_month = str(int(subs_month) - 1)
                    if len(subs_month) == 1:
                        subs_month = '0' + subs_month
                    if subs_month == "00":
                        subs_month = "12"
                    subs_year = str(int(subs_year) - 1)
                    pvs_month = str(int(pvs_month) + 1)
                    if len(pvs_month) == 1:
                        pvs_month = '0' + pvs_month
                    if pvs_month == "13":
                        pvs_month = "01"
                        pvs_year = str(int(pvs_year) + 1)
                    if len(pvs_month) == 1:
                        pvs_month = '0' + pvs_month
                    return pvs_year + '-' + pvs_month + '-' + day + '/' + subs_year + '-' + subs_month + '-' + day
            else:
                if day == "??":                    
                    next_month = str(int(month) + 1)
                    if len(next_month) == 1:
                        next_month = '0' + next_month                    
                    if (pvs_year == subs_year) or ((int(month) >= int(pvs_month)) and (int(month) <= int(subs_month))):
                        if next_month == "13":
                            return pvs_year + '-' + month + '-01/' + str(int(subs_year) + 1) + '-01-01'
                        else:
                            return pvs_year + '-' + month + '-01/' + subs_year + '-' + next_month + '-01'                  
                    elif (int(month) >= int(pvs_month)):
                        subs_year = str(int(subs_year) - 1)
                        if next_month == "13":
                            return pvs_year + '-' + month + '-01/' + str(int(subs_year) + 1) + '-01-01'
                        else:
                            return pvs_year + '-' + month + '-01/' + subs_year + '-' + next_month + '-01'
                    elif (int(month) <= int(subs_month)):
                        pvs_year = str(int(pvs_year) + 1)
                        if next_month == "13":
                            return pvs_year + '-' + month + '-01/' + str(int(subs_year) + 1) + '-01-01'
                        else:
                            return pvs_year + '-' + month + '-01/' + subs_year + '-' + next_month + '-01'
                else:                    
                    if pvs_year == subs_year:
                        return pvs_year + '-' + month + '-' + day
                    else:
                        return pvs_year + '-' + month + '-' + day + '/' + subs_year + '-' + month + '-' + day
        else:
            if month == "??":
                if day == "??":                    
                    if (pvs_year == year) and (subs_year == year):
                        return year + '-' + pvs_month + '-' + pvs_day + '/' + year + '-' + subs_month + '-' + subs_day
                    elif (pvs_year == year):
                        return year + '-' + pvs_month + '-' + pvs_day + '/' + year + '-12-31'
                    elif (subs_year == year):
                        return year + '-01-01/' + year + '-' + subs_month + '-' + subs_day
                    else:
                        return year + '-01-01/' + year + '-12-31'
                else:                    
                    if (pvs_year == year) and (subs_year == year) and (pvs_month == subs_month):
                        return year + '-' + pvs_month + '-' + day
                    elif(pvs_year == year) and (subs_year == year):
                        return year + '-' + pvs_month + '-' + day + '/' + year + '-' + subs_month + '-' + day
                    elif (pvs_year == year) and (pvs_day <= day):
                        return year + '-' + pvs_month + '-' + day + '/' + year + '-12-' + day
                    elif (pvs_year == year):
                        next_month = str(int(pvs_month) + 1)
                        if len(next_month) == 1:
                            next_month = '0' + next_month
                        if next_month == "13":
                            next_month == "12"
                        return year + '-' + next_month + '-' + day + '/' + year + '-12-' + day
                    elif (subs_year == year) and (subs_day >= day):
                        return year + '-01-' + day + '/' + year + '-' + subs_month + '-' + day
                    elif (subs_year == year):
                        subs_month = str(int(subs_month) - 1)
                        if len(subs_month) == 1:
                            subs_month = '0' + subs_month
                        if subs_month == "00":
                            subs_month = "12"                        
                        return year + '-01-' + day + '/' + year + '-' + subs_month + '-' + day
                    else:
                        return year + '-01-' + day + '/' + year + '-12-' + day
                        
            else:
                next_month = str(int(month) + 1)
                if len(next_month) == 1:
                    next_month = '0' + next_month
                if (pvs_year == year) and (pvs_month == month) and (subs_year == year) and (subs_month == month):
                    return year + '-' + month + '-' + pvs_day + '/' + year + '-' + month + '-' + subs_day
                elif (pvs_year == year) and (pvs_month == month):
                    if next_month == "13":
                        return year + '-' + month + '-' + pvs_day + '/' + str(int(year) + 1) + '-01-01'
                    else:
                        return year + '-' + month + '-' + pvs_day + '/' + year + '-' + next_month + '-01'
                elif (subs_year == year) and (subs_month == month):
                    return year + '-' + month + '-01/' + year + '-' + month + '-' + subs_day
                else:
                    if next_month == "13":
                        return year + '-' + month + '-01/' + str(int(year) + 1) + '-01-01'
                    else:
                        return year + '-' + month + '-01/' + year + '-' + next_month + '-01'                

In [ ]:
#no_test

from nbdev.export import notebook2script
notebook2script()

Converted 12-ssda-xml-parser.ipynb.
Converted 31-collate-xml-entities-spans.ipynb.
Converted 33-split-data.ipynb.
Converted 41-generic-framework-for-spacy-training.ipynb.
Converted 42-initial-model.ipynb.
Converted 51-data-preprocessing.ipynb.
Converted 52-unstructured-to-markup.ipynb.
Converted 53-markup-to-spatial-historian.ipynb.
Converted 54-utility-functions.ipynb.
Converted 61-prodigy-output-training-demo.ipynb.
Converted 62-full-model-application-demo.ipynb.
Converted 63-pt-model-training.ipynb.
Converted 64-es-model-training.ipynb.
Converted 65-all-annotations-model-training.ipynb.
Converted 66-es-guatemala-model-training.ipynb.
Converted 67-death-and-birth-records-together.ipynb.
Converted 70-exhaustive-training.ipynb.
Converted 71-relationship-builder.ipynb.
Converted 72-full-volume-processor.ipynb.
